In [ ]:
import pandas as pd
import os
import zipfile
from google.colab import files
import shutil
import re

# --- Step 1: Upload the Excel dump file ---
print("Please upload your Excel dump file (first workbook should have 'Projects' sheet):")
uploaded_mapping = files.upload()
mapping_file_name = list(uploaded_mapping.keys())[0]

# --- Step 2: Read the specific sheet and columns (case-insensitive) ---
df = pd.read_excel(mapping_file_name, sheet_name='Projects')

# Find columns case-insensitively
tender_col = next((col for col in df.columns if 'tender id' in col.lower()), None)
soi_col = next((col for col in df.columns if 'soi' in col.lower()), None)

if not tender_col:
    print("Error: No 'Tender ID'/'Tender Id' column found. Available columns:")
    print(df.columns.tolist())
    exit()

if not soi_col:
    print("Error: No 'SOI' column found. Available columns:")
    print(df.columns.tolist())
    exit()

print(f"Using columns: '{tender_col}' → '{soi_col}'")

# Create mapping: Tender ID → SOI (cleaned values)
tender_id_to_soi = {}
for idx, row in df.iterrows():
    tender_id = str(row[tender_col]).strip()
    soi = str(row[soi_col]).strip()
    tender_id_to_soi[tender_id] = soi

print("Mapping created. Example entries:")
for i, (k, v) in enumerate(list(tender_id_to_soi.items())[:3]):
    print(f"  {k} → {v}")

# --- Step 3: Upload ZIP file containing files to rename ---
print("\nPlease upload your ZIP file containing Tender ID-named files:")
uploaded_zip = files.upload()
zip_filename = list(uploaded_zip.keys())[0]

# --- Step 4: Extract ZIP file to temp folder ---
extract_path = '/content/temp_extracted/'
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("✓ Extracted files to temp folder")

# --- Step 5: Rename files in place ---
renamed_count = 0
skipped_count = 0

print("\n=== Starting mass rename ===")
for filename in os.listdir(extract_path):
    if filename == mapping_file_name or filename == zip_filename:
        continue

    base_name, file_ext = os.path.splitext(filename)

    # Extract Tender ID from filename
    matched_tender_id = None
    for tender_id in tender_id_to_soi.keys():
        if tender_id in base_name:
            matched_tender_id = tender_id
            break

    if matched_tender_id:
        new_soi_name = str(tender_id_to_soi[matched_tender_id])
        new_filename = new_soi_name + file_ext

        old_path = os.path.join(extract_path, filename)
        new_path = os.path.join(extract_path, new_filename)

        if os.path.exists(new_path):
            print(f"⚠ '{new_filename}' already exists, skipping '{filename}'")
            skipped_count += 1
            continue

        try:
            os.rename(old_path, new_path)
            print(f"✓ Renamed '{filename}' → '{new_filename}' (matched: {matched_tender_id})")
            renamed_count += 1
        except OSError as e:
            print(f"✗ Error renaming '{filename}': {e}")
            skipped_count += 1
    else:
        print(f"- '{filename}' - No Tender ID found (skipped)")
        skipped_count += 1

# --- Step 6: Create EAS renamed ZIP and AUTO-DOWNLOAD ---
eas_zip_path = '/content/EAS_renamed.zip'
shutil.make_archive(eas_zip_path.replace('.zip', ''), 'zip', extract_path)

print(f"\n=== SUMMARY ===")
print(f"✅ Successfully renamed: {renamed_count} files")
print(f"⏭️  Skipped: {skipped_count} files")
print(f"📦 Creating 'EAS_renamed.zip' with all renamed files...")

# Trigger automatic download
files.download(eas_zip_path)
print("✅ 'EAS_renamed.zip' is DOWNLOADING automatically to your Downloads folder!")
print("   Contains only the renamed files from your upload.")

# Cleanup temp folder
shutil.rmtree(extract_path)
print("🧹 Cleaned up temporary files.")